In [1]:
import sys
import os
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '../..')))

# Path Configuration
from tools.preprocess import *

# Processing context
trait = "Chronic_obstructive_pulmonary_disease_(COPD)"

# Input paths
tcga_root_dir = "../../input/TCGA"

# Output paths
out_data_file = "../../output/preprocess/Chronic_obstructive_pulmonary_disease_(COPD)/TCGA.csv"
out_gene_data_file = "../../output/preprocess/Chronic_obstructive_pulmonary_disease_(COPD)/gene_data/TCGA.csv"
out_clinical_data_file = "../../output/preprocess/Chronic_obstructive_pulmonary_disease_(COPD)/clinical_data/TCGA.csv"
json_path = "../../output/preprocess/Chronic_obstructive_pulmonary_disease_(COPD)/cohort_info.json"


### Step 1: Initial Data Loading

In [2]:
import os

# Check if there's a suitable cohort directory for COPD
print(f"Looking for a relevant cohort directory for {trait}...")

# Check available cohorts
available_dirs = os.listdir(tcga_root_dir)
print(f"Available cohorts: {available_dirs}")

# COPD-related keywords
copd_related_keywords = ['lung', 'pulmonary', 'respiratory', 'copd', 'bronchitis', 'emphysema', 'airway']

# Look for COPD related directories
copd_related_dirs = []
for d in available_dirs:
    if any(keyword in d.lower() for keyword in copd_related_keywords):
        copd_related_dirs.append(d)

print(f"COPD related cohorts: {copd_related_dirs}")

if not copd_related_dirs:
    print(f"No suitable cohort found for {trait}.")
    # Mark the task as completed by recording the unavailability
    validate_and_save_cohort_info(
        is_final=False,
        cohort="TCGA",
        info_path=json_path,
        is_gene_available=False,
        is_trait_available=False
    )
    # Exit the script early since no suitable cohort was found
    selected_cohort = None
else:
    # For COPD, the general lung cancer cohort would be most relevant
    # Prioritize general lung cancer over specific subtypes if available
    if 'TCGA_Lung_Cancer_(LUNG)' in copd_related_dirs:
        selected_cohort = 'TCGA_Lung_Cancer_(LUNG)'
    else:
        selected_cohort = copd_related_dirs[0]

if selected_cohort:
    print(f"Selected cohort: {selected_cohort}")
    
    # Get the full path to the selected cohort directory
    cohort_dir = os.path.join(tcga_root_dir, selected_cohort)
    
    # Get the clinical and genetic data file paths
    clinical_file_path, genetic_file_path = tcga_get_relevant_filepaths(cohort_dir)
    
    print(f"Clinical data file: {os.path.basename(clinical_file_path)}")
    print(f"Genetic data file: {os.path.basename(genetic_file_path)}")
    
    # Load the clinical and genetic data
    clinical_df = pd.read_csv(clinical_file_path, index_col=0, sep='\t')
    genetic_df = pd.read_csv(genetic_file_path, index_col=0, sep='\t')
    
    # Print the column names of the clinical data
    print("\nClinical data columns:")
    print(clinical_df.columns.tolist())
    
    # Basic info about the datasets
    print(f"\nClinical data shape: {clinical_df.shape}")
    print(f"Genetic data shape: {genetic_df.shape}")


Looking for a relevant cohort directory for Chronic_obstructive_pulmonary_disease_(COPD)...
Available cohorts: ['TCGA_Liver_Cancer_(LIHC)', 'TCGA_Lower_Grade_Glioma_(LGG)', 'TCGA_lower_grade_glioma_and_glioblastoma_(GBMLGG)', 'TCGA_Lung_Adenocarcinoma_(LUAD)', 'TCGA_Lung_Cancer_(LUNG)', 'TCGA_Lung_Squamous_Cell_Carcinoma_(LUSC)', 'TCGA_Melanoma_(SKCM)', 'TCGA_Mesothelioma_(MESO)', 'TCGA_Ocular_melanomas_(UVM)', 'TCGA_Ovarian_Cancer_(OV)', 'TCGA_Pancreatic_Cancer_(PAAD)', 'TCGA_Pheochromocytoma_Paraganglioma_(PCPG)', 'TCGA_Prostate_Cancer_(PRAD)', 'TCGA_Rectal_Cancer_(READ)', 'TCGA_Sarcoma_(SARC)', 'TCGA_Stomach_Cancer_(STAD)', 'TCGA_Testicular_Cancer_(TGCT)', 'TCGA_Thymoma_(THYM)', 'TCGA_Thyroid_Cancer_(THCA)', 'TCGA_Uterine_Carcinosarcoma_(UCS)', '.DS_Store', 'CrawlData.ipynb', 'TCGA_Acute_Myeloid_Leukemia_(LAML)', 'TCGA_Adrenocortical_Cancer_(ACC)', 'TCGA_Bile_Duct_Cancer_(CHOL)', 'TCGA_Bladder_Cancer_(BLCA)', 'TCGA_Breast_Cancer_(BRCA)', 'TCGA_Cervical_Cancer_(CESC)', 'TCGA_Colon_an


Clinical data columns:
['ABSOLUTE_Ploidy', 'ABSOLUTE_Purity', 'AKT1', 'ALK_translocation', 'BRAF', 'CBL', 'CTNNB1', 'Canonical_mut_in_KRAS_EGFR_ALK', 'Cnncl_mt_n_KRAS_EGFR_ALK_RET_ROS1_BRAF_ERBB2_HRAS_NRAS_AKT1_MAP2', 'EGFR', 'ERBB2', 'ERBB4', 'Estimated_allele_fraction_of_a_clonal_varnt_prsnt_t_1_cpy_pr_cll', 'Expression_Subtype', 'HRAS', 'KRAS', 'MAP2K1', 'MET', 'NRAS', 'PIK3CA', 'PTPN11', 'Pathology', 'Pathology_Updated', 'RET_translocation', 'ROS1_translocation', 'STK11', 'WGS_as_of_20120731_0_no_1_yes', '_INTEGRATION', '_PANCAN_CNA_PANCAN_K8', '_PANCAN_Cluster_Cluster_PANCAN', '_PANCAN_DNAMethyl_PANCAN', '_PANCAN_RPPA_PANCAN_K8', '_PANCAN_UNC_RNAseq_PANCAN_K16', '_PANCAN_miRNA_PANCAN', '_PANCAN_mutation_PANCAN', '_PATIENT', '_cohort', '_primary_disease', '_primary_site', 'additional_pharmaceutical_therapy', 'additional_radiation_therapy', 'additional_surgery_locoregional_procedure', 'additional_surgery_metastatic_procedure', 'age_at_initial_pathologic_diagnosis', 'anatomic_neopla

### Step 2: Find Candidate Demographic Features

In [3]:
# Identify candidate columns for age
candidate_age_cols = ['age_at_initial_pathologic_diagnosis', 'days_to_birth']

# Identify candidate columns for gender
candidate_gender_cols = ['gender']

# Load the clinical data file
cohort_dir = os.path.join(tcga_root_dir, 'TCGA_Lung_Cancer_(LUNG)')
clinical_file_path, _ = tcga_get_relevant_filepaths(cohort_dir)
clinical_df = pd.read_csv(clinical_file_path, sep='\t', index_col=0)

# Extract and preview age columns
if candidate_age_cols:
    age_preview = {col: clinical_df[col].head(5).tolist() for col in candidate_age_cols}
    print("Age columns preview:")
    print(age_preview)

# Extract and preview gender columns
if candidate_gender_cols:
    gender_preview = {col: clinical_df[col].head(5).tolist() for col in candidate_gender_cols}
    print("Gender columns preview:")
    print(gender_preview)


Age columns preview:
{'age_at_initial_pathologic_diagnosis': [70.0, 67.0, 79.0, 68.0, 66.0], 'days_to_birth': [-25752.0, -24532.0, -29068.0, -24868.0, -24411.0]}
Gender columns preview:
{'gender': ['MALE', 'MALE', 'FEMALE', 'MALE', 'MALE']}


### Step 3: Select Demographic Features

In [4]:
# Inspect the age columns
print("Age columns evaluation:")
for col, values in {'age_at_initial_pathologic_diagnosis': [70.0, 67.0, 79.0, 68.0, 66.0], 'days_to_birth': [-25752.0, -24532.0, -29068.0, -24868.0, -24411.0]}.items():
    non_null_count = sum(1 for v in values if v is not None)
    print(f"  {col}: {non_null_count}/5 non-null values")
    if col == 'age_at_initial_pathologic_diagnosis':
        print(f"    Values represent age in years directly: {values}")
    elif col == 'days_to_birth':
        print(f"    Values represent negative days from birth (needs conversion): {values}")

# Inspect the gender columns
print("\nGender columns evaluation:")
for col, values in {'gender': ['MALE', 'MALE', 'FEMALE', 'MALE', 'MALE']}.items():
    non_null_count = sum(1 for v in values if v is not None and v != '')
    print(f"  {col}: {non_null_count}/5 non-null values, values: {values}")

# Select appropriate columns
age_col = "age_at_initial_pathologic_diagnosis"  # Direct age is easier to work with than days_to_birth
gender_col = "gender"  # Only one option available and it has valid values

# Print chosen columns
print("\nChosen demographic columns:")
print(f"Age column: {age_col}")
print(f"Gender column: {gender_col}")


Age columns evaluation:
  age_at_initial_pathologic_diagnosis: 5/5 non-null values
    Values represent age in years directly: [70.0, 67.0, 79.0, 68.0, 66.0]
  days_to_birth: 5/5 non-null values
    Values represent negative days from birth (needs conversion): [-25752.0, -24532.0, -29068.0, -24868.0, -24411.0]

Gender columns evaluation:
  gender: 5/5 non-null values, values: ['MALE', 'MALE', 'FEMALE', 'MALE', 'MALE']

Chosen demographic columns:
Age column: age_at_initial_pathologic_diagnosis
Gender column: gender


### Step 4: Feature Engineering and Validation

In [5]:
# 1. Extract and standardize clinical features
# Use tcga_select_clinical_features which will automatically create the trait variable and add age/gender if provided
cohort_dir = os.path.join(tcga_root_dir, 'TCGA_Lung_Cancer_(LUNG)')
clinical_file_path, genetic_file_path = tcga_get_relevant_filepaths(cohort_dir)

# Load the clinical data if not already loaded
clinical_df = pd.read_csv(clinical_file_path, sep='\t', index_col=0)

linked_clinical_df = tcga_select_clinical_features(
    clinical_df, 
    trait=trait, 
    age_col=age_col, 
    gender_col=gender_col
)

# Print preview of clinical features
print("Clinical features (first 5 rows):")
print(linked_clinical_df.head())

# 2. Process gene expression data
print("\nProcessing gene expression data...")
# Load genetic data from the same cohort directory
genetic_df = pd.read_csv(genetic_file_path, sep='\t', index_col=0)

# Check gene data shape
print(f"Original gene data shape: {genetic_df.shape}")

# Save a version of the gene data before normalization (as a backup)
os.makedirs(os.path.dirname(out_gene_data_file), exist_ok=True)
genetic_df.to_csv(out_gene_data_file.replace('.csv', '_original.csv'))

# We need to transpose genetic data so genes are rows and samples are columns for normalization
gene_df_for_norm = genetic_df.copy()  # Keep original orientation for now

# Try to normalize gene symbols - adding debug output to understand what's happening
print("Attempting to normalize gene symbols...")
try:
    # First check if we need to transpose based on the data format
    # In TCGA data, typically genes are rows and samples are columns
    if gene_df_for_norm.shape[0] > gene_df_for_norm.shape[1]:
        # More rows than columns, likely genes are rows already
        normalized_gene_df = normalize_gene_symbols_in_index(gene_df_for_norm)
    else:
        # Need to transpose first
        normalized_gene_df = normalize_gene_symbols_in_index(gene_df_for_norm.T)
        
    print(f"Gene data shape after normalization: {normalized_gene_df.shape}")
    
    # Check if normalization returned empty DataFrame
    if normalized_gene_df.shape[0] == 0:
        print("WARNING: Gene symbol normalization returned an empty DataFrame.")
        print("Using original gene data instead of normalized data.")
        # Use original data
        normalized_gene_df = genetic_df
    
except Exception as e:
    print(f"Error during gene symbol normalization: {e}")
    print("Using original gene data instead.")
    normalized_gene_df = genetic_df

# Save gene data
normalized_gene_df.to_csv(out_gene_data_file)
print(f"Gene data saved to: {out_gene_data_file}")

# 3. Link clinical and genetic data
# TCGA data uses the same sample IDs in both datasets
print("\nLinking clinical and genetic data...")
print(f"Clinical data shape: {linked_clinical_df.shape}")
print(f"Genetic data shape: {normalized_gene_df.shape}")

# Find common samples between clinical and genetic data
# In TCGA, samples are typically columns in the gene data and index in the clinical data
common_samples = set(linked_clinical_df.index).intersection(set(normalized_gene_df.columns))
print(f"Number of common samples: {len(common_samples)}")

if len(common_samples) == 0:
    print("ERROR: No common samples found between clinical and genetic data.")
    # Try the alternative orientation
    common_samples = set(linked_clinical_df.index).intersection(set(normalized_gene_df.index))
    print(f"Checking alternative orientation: {len(common_samples)} common samples found.")
    
    if len(common_samples) == 0:
        # Use is_final=False mode which doesn't require df and is_biased
        validate_and_save_cohort_info(
            is_final=False,
            cohort="TCGA",
            info_path=json_path,
            is_gene_available=True,
            is_trait_available=True
        )
        print("The dataset was determined to be unusable for this trait due to no common samples. No data files were saved.")
else:
    # Filter clinical data to only include common samples
    linked_clinical_df = linked_clinical_df.loc[list(common_samples)]
    
    # Create linked data by merging
    linked_data = pd.concat([linked_clinical_df, normalized_gene_df[list(common_samples)].T], axis=1)
    
    print(f"\nLinked data shape: {linked_data.shape}")
    print("Linked data preview (first 5 rows, first few columns):")
    display_cols = [trait, 'Age', 'Gender'] + list(linked_data.columns[3:5])
    print(linked_data[display_cols].head())
    
    # 4. Handle missing values
    linked_data = handle_missing_values(linked_data, trait)
    print(f"\nData shape after handling missing values: {linked_data.shape}")
    
    # 5. Check for bias in trait and demographic features
    print("\nChecking for bias in features:")
    is_trait_biased, linked_data = judge_and_remove_biased_features(linked_data, trait)
    
    # 6. Validate and save cohort info
    print("\nPerforming final validation...")
    is_usable = validate_and_save_cohort_info(
        is_final=True,
        cohort="TCGA",
        info_path=json_path,
        is_gene_available=len(linked_data.columns) > 3,  # More than just trait/age/gender columns
        is_trait_available=trait in linked_data.columns,
        is_biased=is_trait_biased,
        df=linked_data,
        note="Data from TCGA Lung Cancer cohort used for COPD gene expression analysis."
    )
    
    # 7. Save linked data if usable
    if is_usable:
        os.makedirs(os.path.dirname(out_data_file), exist_ok=True)
        linked_data.to_csv(out_data_file)
        print(f"Linked data saved to: {out_data_file}")
        
        # Also save clinical data separately
        os.makedirs(os.path.dirname(out_clinical_data_file), exist_ok=True)
        clinical_columns = [col for col in linked_data.columns if col in [trait, 'Age', 'Gender']]
        linked_data[clinical_columns].to_csv(out_clinical_data_file)
        print(f"Clinical data saved to: {out_clinical_data_file}")
    else:
        print("The dataset was determined to be unusable for this trait. No data files were saved.")

Clinical features (first 5 rows):
                 Chronic_obstructive_pulmonary_disease_(COPD)   Age  Gender
sampleID                                                                   
TCGA-05-4244-01                                             1  70.0     1.0
TCGA-05-4249-01                                             1  67.0     1.0
TCGA-05-4250-01                                             1  79.0     0.0
TCGA-05-4382-01                                             1  68.0     1.0
TCGA-05-4384-01                                             1  66.0     1.0

Processing gene expression data...


Original gene data shape: (20530, 1129)


Attempting to normalize gene symbols...


Gene data shape after normalization: (19848, 1129)


Gene data saved to: ../../output/preprocess/Chronic_obstructive_pulmonary_disease_(COPD)/gene_data/TCGA.csv

Linking clinical and genetic data...
Clinical data shape: (1299, 3)
Genetic data shape: (19848, 1129)
Number of common samples: 1129

Linked data shape: (1129, 19851)
Linked data preview (first 5 rows, first few columns):
                 Chronic_obstructive_pulmonary_disease_(COPD)   Age  Gender  \
TCGA-78-7147-01                                             1  67.0     0.0   
TCGA-55-7227-01                                             1  77.0     1.0   
TCGA-44-7661-01                                             1  69.0     0.0   
TCGA-85-A4PA-01                                             1  61.0     1.0   
TCGA-38-A44F-01                                             1  80.0     1.0   

                     A1BG  A1BG-AS1  
TCGA-78-7147-01  0.034026  0.522917  
TCGA-55-7227-01 -1.308274 -0.304083  
TCGA-44-7661-01 -0.353574  0.199017  
TCGA-85-A4PA-01  1.474026  2.069817  
TCGA


Data shape after handling missing values: (1129, 19851)

Checking for bias in features:
For the feature 'Chronic_obstructive_pulmonary_disease_(COPD)', the least common label is '0' with 110 occurrences. This represents 9.74% of the dataset.
The distribution of the feature 'Chronic_obstructive_pulmonary_disease_(COPD)' in this dataset is fine.

Quartiles for 'Age':
  25%: 60.0
  50% (Median): 67.0
  75%: 73.0
Min: 38.0
Max: 90.0
The distribution of the feature 'Age' in this dataset is fine.

For the feature 'Gender', the least common label is '0.0' with 455 occurrences. This represents 40.30% of the dataset.
The distribution of the feature 'Gender' in this dataset is fine.


Performing final validation...


Linked data saved to: ../../output/preprocess/Chronic_obstructive_pulmonary_disease_(COPD)/TCGA.csv
Clinical data saved to: ../../output/preprocess/Chronic_obstructive_pulmonary_disease_(COPD)/clinical_data/TCGA.csv
